# Create lighter dataset in json format for usage in viz

In [1]:
import pandas as pd
import numpy as np
import json
# imprt data from the large dataset
data_path = "data/aggregated_parlamentarier_nested.json"
df = pd.read_json(data_path)

In [3]:
data_branche = pd.DataFrame(columns=['branche', 'sub_branche', 'party', 'name', 'wirksamkeit', 'lobby_name', 'detail_year', 'detail_geld', 'detail_beschreibung']) #define what we want as fields
for row in range(len(df)):
    for i in range(len(df.iloc[row]['interessenbindungen'])):
        if df.iloc[row]['interessenbindungen'][i]['organisation']!=None:
            for j in range(len(df.iloc[row]['interessenbindungen'][i]['organisation']['interessengruppe'])):
                years = []
                beschreibung = []
                geld_per_year = []
                # Get these fields  
                branche = df.iloc[row]['interessenbindungen'][i]['organisation']['interessengruppe'][j]['branche']['anzeige_name_fr']
                sub_branche = df.iloc[row]['interessenbindungen'][i]['organisation']['interessengruppe'][j]['anzeige_name_fr']
                party = df.iloc[row]['partei_name']
                name = df.iloc[row]['name_fr']
                wirksamkeit = df.iloc[row]['interessenbindungen'][i]['wirksamkeit']
                wirksamkeit_nb = 0
                lobby_name = df.iloc[row]['interessenbindungen'][i]['organisation']['name']
                match wirksamkeit: #transform wirksamkeit to int
                    case 'tief':
                        wirksamkeit_nb=3
                    case 'mittel':
                        wirksamkeit_nb=1
                    case 'hoch':
                        wirksamkeit_nb=2
                for x in range(len(df.iloc[row]['interessenbindungen'][i]['verguetungen'])): #sum money received
                    #geld+=df.iloc[row]['interessenbindungen'][i]['verguetungen'][x]['verguetung']
                    geld_per_year.append(df.iloc[row]['interessenbindungen'][i]['verguetungen'][x]['verguetung'])
                    years.append(df.iloc[row]['interessenbindungen'][i]['verguetungen'][x]['jahr'])
                    beschreibung.append(df.iloc[row]['interessenbindungen'][i]['verguetungen'][x]['beschreibung'])
                data_branche.loc[len(data_branche)]= [branche, sub_branche, party, name, wirksamkeit_nb, lobby_name, years, geld_per_year, beschreibung]

## Intermediate results

In [4]:
display(data_branche)

,branche,sub_branche,party,name,wirksamkeit,lobby_name,detail_year,detail_geld,detail_beschreibung
0,Politique intérieure / Economie publique,Cantons/Régions,Schweizerische Volkspartei,Thomas Jan de Courten,3,Komitee für ein selbständiges Baselbiet,"[2020, 2021, 2022, 2023, 2024]","[0, 0, 0, 0, 0]","[Ehrenamtlich, Ehrenamtlich, Ehrenamtlich, Ehr..."
1,Transport,Logistique/Transport,Schweizerische Volkspartei,Thomas Jan de Courten,1,SPEDLOGSWISS Verband schweizerischer Spedition...,"[2020, 2021, 2022, 2023, 2024]","[1, 1, 1, 1, 1]","[None, None, None, None, None]"
2,Politique intérieure / Economie publique,Politique en général,Schweizerische Volkspartei,Thomas Jan de Courten,1,Schweizerischer Verband der Bürgergemeinden un...,"[2020, 2021, 2022, 2023, 2024]","[0, 0, 0, 0, 0]","[Ehrenamtlich, Ehrenamtlich, Ehrenamtlich, Ehr..."
3,Économie,Fonds de pension et de retraite,Schweizerische Volkspartei,Thomas Jan de Courten,1,Vorsorgeforum,[2024],[1],[Bezahlt]
4,Économie,Biotechnologie,Schweizerische Volkspartei,Thomas Jan de Courten,3,IG Biomedizinische Forschung und Innovation,"[2020, 2021, 2022, 2023, 2024]","[1, 1, 1, 1, 1]","[None, None, None, None, None]"
...,...,...,...,...,...,...,...,...,...
7128,Sport,Sport populaire,Sozialdemokratische Partei,Andrea Zryd,3,Panathlon- Club Biel,[2024],[-1],[Bezahlendes Mitglied]
7129,Sport,Sport populaire,Sozialdemokratische Partei,Andrea Zryd,3,Bernischer Verband für Sport in der Schule,[2024],[-1],[Bezahlendes Mitglied]
7130,Sport,Sport d'élite,Sozialdemokratische Partei,Andrea Zryd,3,Zryd & Reinhard GmbH,[2024],[1],[Bezahlt]
7131,Sécurité sociale,Enfants/Jeunesse,Sozialdemokratische Partei,Andrea Zryd,3,"Swiss Academy for Development, SA4D",[2024],[0],[Ehrenamtlich]


In [5]:
# Assuming 'data_branche' is your DataFrame
data_branche2 = data_branche.groupby(by=["branche", "sub_branche", "party", "name"]).agg(
    wirksamkeit=('wirksamkeit', 'sum'),
    lobby_name=('lobby_name', lambda x: list(x)),
    years=('detail_year', lambda x: list(x)),
    geld_per_year=('detail_geld', lambda x: list(x)),
    bescreibung=('detail_beschreibung', lambda x: list(x))
).reset_index()

# If you want to rename the columns to be more descriptive
data_branche2.columns = [
    'branche', 'sub_branche', 'party', 'name', 'total_wirksamkeit',
    'lobby_name', 'detail_year', 'detail_geld', 'detail_beschreibung'
]

print(data_branche2)


          branche                     sub_branche  \
0     Agriculture          Agriculture en général   
1     Agriculture          Agriculture en général   
2     Agriculture          Agriculture en général   
3     Agriculture          Agriculture en général   
4     Agriculture          Agriculture en général   
...           ...                             ...   
4711      Énergie           Fournisseur d'énergie   
4712      Énergie           Fournisseur d'énergie   
4713      Énergie           Fournisseur d'énergie   
4714      Énergie  Industrie du gaz et du pétrole   
4715      Énergie  Industrie du gaz et du pétrole   

                                party                           name  \
0                           Die Mitte                  Andreas Meier   
1                           Die Mitte                Benjamin Roduit   
2                           Die Mitte  Brigitte Maria Häberli-Koller   
3                           Die Mitte               Charles Juillard   
4  

In [6]:
data_branche = data_branche2

## Write as dict the data in the a particular form

In [12]:
list_branche = []
for branche in data_branche['branche'].unique():
    list_sb = []
    for sous_branche in data_branche[data_branche['branche'] == branche]['sub_branche'].unique():
        list_party = []
        for party in data_branche[(data_branche['branche'] == branche) & (data_branche['sub_branche'] == sous_branche)]['party'].unique():
            list_people = []
            for index, row in data_branche[(data_branche['branche'] == branche) & (data_branche['sub_branche'] == sous_branche) & (data_branche['party']==party)].iterrows():
                list_people.append({'name':row['name'], 'wirksamkeit':row['total_wirksamkeit'], 'party':row['party'], 'sub_branche':row['sub_branche'], 'branche':row['branche'], 'lobby_name':row['lobby_name'], 'detail_year':row['detail_year'], 'detail_geld':row['detail_year'], 'detail_beschreibung':row['detail_beschreibung']})
            list_party.append({'name':party, 'children':list_people, 'type':'party'})
        list_sb.append({'name':sous_branche, 'children':list_party, 'type':'subbranche'})
    list_branche.append({'name':branche,'children':list_sb, 'type':'branche'})
final_json = {'name': "lobby", 'children': list_branche}

In [14]:
# Write as json the dict
with open("data_final.json", "w") as final:
    json.dump(final_json, final)

# Get the non-parlementary people that have access through invitation of parlementary members

In [28]:
party = {}
for el in df['partei_name'].unique():
    party[el]=1

In [29]:
data_branche = pd.DataFrame(columns=['branche', 'sub_branche', 'party', 'name', 'wirksamkeit', 'lobby_name', 'detail_year', 'detail_geld', 'detail_beschreibung']) #define what we want as fields
for row in range(len(df)):
    for i in range(len(df.iloc[row]['zutrittsberechtigungen'])):
        if len(df.iloc[row]['zutrittsberechtigungen'][i]['mandate'])>0:
            party[df.iloc[row]['partei_name']]=party[df.iloc[row]['partei_name']]+1
            

See if some parties use their invite right more than others

In [32]:
for key, nb in party.items():
    print(f"Party {key} has {round(df['partei_name'].value_counts()[key]/df.shape[0], 2)} of the members but {round(nb/sum(party.values()),2)} of the invites")

Party Schweizerische Volkspartei has 0.28 of the members but 0.23 of the invites
Party Die Mitte has 0.18 of the members but 0.22 of the invites
Party Freisinnig-Demokratische Partei has 0.15 of the members but 0.12 of the invites
Party Sozialdemokratische Partei has 0.2 of the members but 0.18 of the invites
Party Grünliberale Partei has 0.04 of the members but 0.03 of the invites
Party Grüne Partei der Schweiz has 0.1 of the members but 0.11 of the invites
Party Mouvement Citoyens Romands has 0.01 of the members but 0.02 of the invites
Party Lega dei Ticinesi has 0.0 of the members but 0.02 of the invites
Party Basels starke Alternative has 0.0 of the members but 0.02 of the invites
Party Evangelische Volkspartei has 0.01 of the members but 0.02 of the invites
Party Eidgenössisch-Demokratische Union has 0.01 of the members but 0.02 of the invites
Party Alternative - die Grünen Zug has 0.0 of the members but 0.02 of the invites
Party Liberal-Demokratische Partei has 0.0 of the members

## Make a dataset using the invited people that have a mandate as we did with the parlementy members

In [33]:
data_branche_invited = pd.DataFrame(columns=['branche', 'sub_branche', 'party', 'name_inviting', 'name_invited', 'lobby_name']) #define what we want as fields
for row in range(len(df)):
    for i in range(len(df.iloc[row]['zutrittsberechtigungen'])):
        if len(df.iloc[row]['zutrittsberechtigungen'][i]['mandate'])>0:
            for j in range(len(df.iloc[row]['zutrittsberechtigungen'][i]['mandate'])):
                for m in range(len(df.iloc[row]['zutrittsberechtigungen'][i]['mandate'][j]['organisation']['interessengruppe'])):
                    # Get these fields  
                    branche = df.iloc[row]['zutrittsberechtigungen'][i]['mandate'][j]['organisation']['interessengruppe'][m]['branche']['anzeige_name_fr']
                    sub_branche = df.iloc[row]['zutrittsberechtigungen'][i]['mandate'][j]['organisation']['interessengruppe'][m]['anzeige_name_fr']
                    party = df.iloc[row]['partei_name']
                    name_inviting = df.iloc[row]['name_fr']
                    name_invited = df.iloc[row]['zutrittsberechtigungen'][i]['name_fr']
                    lobby_name = df.iloc[row]['zutrittsberechtigungen'][i]['mandate'][j]['organisation']['name']
                    data_branche_invited.loc[len(data_branche_invited)]= [branche, sub_branche, party, name_inviting, name_invited, lobby_name]

In [34]:
# Assuming 'data_branche' is your DataFrame
data_branche_invited_2 = data_branche_invited.groupby(by=["branche", "sub_branche", "party", "name_invited", "name_inviting"]).agg(
    lobby_name=('lobby_name', lambda x: list(x)),
).reset_index()


print(data_branche_invited_2)
data_branche_invited = data_branche_invited_2


           branche             sub_branche                            party  \
0      Agriculture  Agriculture écologique         Grüne Partei der Schweiz   
1      Agriculture     Production/Commerce       Schweizerische Volkspartei   
2    Communication       Médias en général       Schweizerische Volkspartei   
3    Communication       Médias en général       Sozialdemokratische Partei   
4    Communication         Médias imprimés                        Die Mitte   
..             ...                     ...                              ...   
181        Énergie          Anti-nucléaire                        Die Mitte   
182        Énergie          Anti-nucléaire  Freisinnig-Demokratische Partei   
183        Énergie          Anti-nucléaire       Sozialdemokratische Partei   
184        Énergie       Energie nucléaire                        Die Mitte   
185        Énergie   Energies alternatives                        Die Mitte   

             name_invited                 name_invi

In [35]:
list_branche_invited = []
for branche in data_branche_invited['branche'].unique():
    list_sb_invited = []
    for sous_branche in data_branche_invited[data_branche_invited['branche'] == branche]['sub_branche'].unique():
        list_party_invited = []
        for party in data_branche_invited[(data_branche_invited['branche'] == branche) & (data_branche_invited['sub_branche'] == sous_branche)]['party'].unique():
            list_people_invited = []
            for index, row in data_branche_invited[(data_branche_invited['branche'] == branche) & (data_branche_invited['sub_branche'] == sous_branche) & (data_branche_invited['party']==party)].iterrows():
                list_people_invited.append({'name_invited':row['name_invited'], 'name_inviting':row['name_inviting'], 'party':row['party'], 'sub_branche':row['sub_branche'], 'branche':row['branche'], 'lobby_name':row['lobby_name']})
            list_party_invited.append({'name':party, 'children':list_people_invited, 'type':'party'})
        list_sb_invited.append({'name':sous_branche, 'children':list_party_invited, 'type':'subbranche'})
    list_branche_invited.append({'name':branche,'children':list_sb_invited, 'type':'branche'})
final_json_invited = {'name': "lobby", 'children': list_branche_invited}

In [36]:
# Write as json the dict
with open("data_final_invited.json", "w") as final:
    json.dump(final_json_invited, final)